<a href="https://colab.research.google.com/github/Gangyao-01/Subjectivity_analyzing/blob/master/%E4%BD%BF%E7%94%A8MLP%E5%AF%B9%E6%96%87%E6%9C%AC%E5%AE%A2%E8%A7%82%E6%80%A7%E5%88%86%E6%9E%90%E7%9A%84%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E5%B0%9D%E8%AF%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
from sklearn.metrics import confusion_matrix, f1_score

In [0]:
from sklearn import linear_model

In [0]:
df = pd.read_csv('attributes.csv')

In [0]:
df.head()                                 #返回前五个数据 

,TextID,URL,Label,totalWordsCount,semanticobjscore,semanticsubjscore,CC,CD,DT,EX,FW,INs,JJ,JJR,JJS,LS,MD,NN,NNP,NNPS,NNS,PDT,POS,PRP,PRPS,RB,RBR,RBS,RP,SYM,TOs,UH,VB,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WPS,WRB,baseform,Quotes,questionmarks,exclamationmarks,fullstops,commas,semicolon,colon,ellipsis,pronouns1st,pronouns2nd,pronouns3rd,compsupadjadv,past,imperative,present3rd,present1st2nd,sentence1st,sentencelast,txtcomplexity
0,Text0001,http://msn.foxsports.com/foxsoccer/mexico/stor...,1,109,0,1,7,9,0,5,8,6,0,0,0,0,29,0,0,12,0,0,1,2,2,0,0,2,0,3,0,0,11,0,2,0,0,0,1,0,1,0,2,0,0,0,4,2,0,0,0,0,0,3,0,11,0,0,0,0,1,18
1,Text0002,http://msn.foxsports.com/foxsoccer/premierleag...,1,309,21,4,1,19,1,4,35,23,0,0,0,2,93,1,0,17,0,0,7,5,16,0,0,0,1,3,0,0,13,7,13,9,14,0,0,0,3,0,8,7,0,0,19,3,0,5,0,1,0,10,0,13,0,14,9,1,1,14
2,Text0003,http://uk.eurosport.yahoo.com/04022011/58/fed-...,1,149,6,1,8,14,0,5,15,11,0,0,0,0,47,1,0,4,0,0,4,0,7,0,0,1,0,4,0,0,8,2,2,2,3,0,0,0,2,0,4,0,0,0,6,3,0,0,0,0,0,2,0,8,0,3,2,1,1,18
3,Text0004,http://uk.eurosport.yahoo.com/07022011/58/bund...,1,305,18,5,7,26,0,10,37,21,1,1,0,0,83,2,0,18,0,0,6,2,9,1,0,1,2,9,0,1,13,12,6,1,7,1,1,0,1,0,5,3,0,0,13,17,0,0,0,0,0,8,3,13,1,7,1,1,1,20
4,Text0005,http://uk.eurosport.yahoo.com/05022011/58/fed-...,1,491,23,8,33,47,0,12,61,36,0,1,0,2,142,1,0,15,0,0,7,9,10,1,0,1,0,13,0,1,34,5,6,6,5,0,3,0,1,0,7,4,0,0,18,17,0,0,0,0,0,16,2,34,1,5,6,1,1,24


In [0]:
df.columns                                  #数据框索引列表

Index(['TextID', 'URL', 'Label', 'totalWordsCount', 'semanticobjscore',
       'semanticsubjscore', 'CC', 'CD', 'DT', 'EX', 'FW', 'INs', 'JJ', 'JJR',
       'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP',
       'PRPS', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TOs', 'UH', 'VB', 'VBD',
       'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPS', 'WRB', 'baseform',
       'Quotes', 'questionmarks', 'exclamationmarks', 'fullstops', 'commas',
       'semicolon', 'colon', 'ellipsis', 'pronouns1st', 'pronouns2nd',
       'pronouns3rd', 'compsupadjadv', 'past', 'imperative', 'present3rd',
       'present1st2nd', 'sentence1st', 'sentencelast', 'txtcomplexity'],
      dtype='object')

In [0]:
!pip install -q tensorflow==2.0.0-alpha0     #安装tensorflow2.0alpha版

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow import feature_column

In [0]:
tf.random.set_seed(1)                                                    #设定随机种子值保证运行结果不变

In [0]:
train, test = train_test_split(df, test_size=0.2, random_state=1)        #划分数据集为训练集和测试集，比例为80：20

In [0]:
train, valid = train_test_split(train, test_size=0.2, random_state=1)    #划分训练集集为最终的训练集和验证集，比例为80：20

In [0]:
#验证划分好的数据长度
print(len(train))
print(len(valid))
print(len(test))

640
160
200


In [0]:
feature_columns = []          #创建一个特征列表

In [0]:
#数字特征列表
numeric_columns = ['semanticobjscore', 'semanticsubjscore',	'CC', 'CD',	'DT', 'EX',	'FW', 'INs', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT',	'POS', 'PRP', 'PRPS', 'RB',	'RBR', 'RBS', 'RP',	'SYM', 'TOs', 'UH',	'VB', 'VBD', 'VBG',	'VBN', 'VBP', 'VBZ', 'WDT',	'WP', 'WPS', 'WRB',	'baseform',	'Quotes', 'questionmarks', 'exclamationmarks', 'fullstops',	'commas', 'semicolon', 'colon',  'ellipsis', 'pronouns1st',	'pronouns2nd', 'pronouns3rd', 'compsupadjadv', 'past', 'imperative', 'present3rd', 'present1st2nd', 'sentence1st', 'sentencelast', 'txtcomplexity']

In [0]:
#将数字特征列表加入空的特征列表中
for header in numeric_columns:
  feature_columns.append(feature_column.numeric_column(header))

In [0]:
#查看特征列表
feature_columns

[NumericColumn(key='semanticobjscore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='semanticsubjscore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='CC', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='CD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='DT', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='FW', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='INs', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='JJ', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='JJR', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericC

In [0]:
from tensorflow.keras import layers

In [0]:
#创建特征层Densefeatures,输入为特征列表
feature_layer = layers.DenseFeatures(feature_columns)

In [0]:
#该函数的功能是将pandas数据框转化为tensorflow数据流
def df_to_tfdata(df, shuffle=True, bs=32):
  df = df.copy()
  labels = df.pop('Label')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df), seed=1)
  ds = ds.batch(bs)
  return ds

In [0]:
#将分好的数据转化为数据流
train_ds = df_to_tfdata(train)
valid_ds = df_to_tfdata(valid, shuffle=False)
test_ds = df_to_tfdata(test, shuffle=False)

In [0]:
#使用keras建立贯序模型，特征层作为输入层，隐藏层有100个节点，使用relu激活函数，输出层为1个神经元，使用sigmoid激活函数
model = keras.Sequential([
  feature_layer,
  layers.Dense(100, activation='relu'),
  layers.Dense(100, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

In [0]:
#配置模型，使用adam优化器，二分类使用binary_crossentropy损失函数，评估参数为准确度
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
#把训练集和验证集放入构建好的模型，训练5个轮次
model.fit(train_ds,
          validation_data=valid_ds,
          epochs=5)

Epoch 1/5
20/20 [==============================] - 2s 102ms/step - loss: 1.6918 - accuracy: 0.5469 - val_loss: 0.8406 - val_accuracy: 0.7937
Epoch 2/5
20/20 [==============================] - 1s 54ms/step - loss: 0.7199 - accuracy: 0.7944 - val_loss: 0.6539 - val_accuracy: 0.8313
Epoch 3/5
20/20 [==============================] - 1s 54ms/step - loss: 0.5633 - accuracy: 0.8181 - val_loss: 0.5363 - val_accuracy: 0.8375
Epoch 4/5
20/20 [==============================] - 1s 54ms/step - loss: 0.5258 - accuracy: 0.8104 - val_loss: 0.5238 - val_accuracy: 0.8313
Epoch 5/5
20/20 [==============================] - 1s 54ms/step - loss: 0.4457 - accuracy: 0.8502 - val_loss: 0.4837 - val_accuracy: 0.8250


In [0]:
#查看模型
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_7 (DenseFeatu multiple                  0         
_________________________________________________________________
dense_21 (Dense)             multiple                  5900      
_________________________________________________________________
dense_22 (Dense)             multiple                  10100     
_________________________________________________________________
dense_23 (Dense)             multiple                  101       
Total params: 16,101
Trainable params: 16,101
Non-trainable params: 0
_________________________________________________________________


In [0]:
#使用测试集输出f1_score
f1_score(test.Label, np.rint(model.predict(test_ds)), average='micro')

0.865

In [0]:
#打印测试集正确率
model.evaluate(test_ds)

7/7 [==============================] - 0s 36ms/step - loss: 0.4340 - accuracy: 0.8650


[0.433998167514801, 0.865]

In [0]:
#输出测试集的混淆矩阵
confusion_matrix(test.Label, np.rint(model.predict(test_ds)))

array([[ 55,  17],
       [ 10, 118]])